In [36]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [37]:
df = pd.read_csv(
    Path('updated_player_data.csv')
)
df.head()

,ID,Name,Age,Nationality,Club,Wage,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,...,Position_RB,Position_RCB,Position_RCM,Position_RDM,Position_RF,Position_RM,Position_RS,Position_RW,Position_RWB,Position_ST
0,158023,L. Messi,31,Argentina,FC Barcelona,565000.0,0,5.0,4.0,4.0,...,False,False,False,False,True,False,False,False,False,False
1,20801,Cristiano Ronaldo,33,Portugal,Juventus,405000.0,1,5.0,4.0,5.0,...,False,False,False,False,False,False,False,False,False,True
2,190871,Neymar Jr,26,Brazil,Paris Saint-Germain,290000.0,1,5.0,5.0,5.0,...,False,False,False,False,False,False,False,False,False,False
3,193080,De Gea,27,Spain,Manchester United,260000.0,1,4.0,3.0,1.0,...,False,False,False,False,False,False,False,False,False,False
4,192985,K. De Bruyne,27,Belgium,Manchester City,355000.0,1,4.0,5.0,4.0,...,False,False,True,False,False,False,False,False,False,False


In [38]:
#Preparing the data
df2 = df.drop(['ID','Name','Nationality','Club','WorkRate'],axis=1)
df2.head()

,Age,Wage,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,BodyType,Height,Weight,Crossing,...,Position_RB,Position_RCB,Position_RCM,Position_RDM,Position_RF,Position_RM,Position_RS,Position_RW,Position_RWB,Position_ST
0,31,565000.0,0,5.0,4.0,4.0,1,67.0,159.0,84.0,...,False,False,False,False,True,False,False,False,False,False
1,33,405000.0,1,5.0,4.0,5.0,1,74.0,183.0,84.0,...,False,False,False,False,False,False,False,False,False,True
2,26,290000.0,1,5.0,5.0,5.0,1,69.0,150.0,79.0,...,False,False,False,False,False,False,False,False,False,False
3,27,260000.0,1,4.0,3.0,1.0,0,76.0,168.0,17.0,...,False,False,False,False,False,False,False,False,False,False
4,27,355000.0,1,4.0,5.0,4.0,1,71.0,154.0,93.0,...,False,False,True,False,False,False,False,False,False,False


In [42]:
# Splitting the data
y = df2['Wage']
X = df2.drop(columns='Wage')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [43]:
# Scaling the data
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# KNN 5

In [ ]:
# Constructing training model
n_neighbors=5
knn5=KNeighborsRegressor(n_neighbors,weights='uniform')
knn5.fit(X_train_scaled,y_train)
y1_knn5=knn5.predict(X_train_scaled)
y1_knn5=list(y1_knn5)

In [ ]:
# Evaluating model on training data
from sklearn.metrics import mean_squared_error
import numpy as np
mse = mean_squared_error(y_train, y1_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_train_scaled,y_train)}")

The RMSE is: 11686.909164119847
The R-squared is 0.7082642532645084


In [48]:
# Using model on test data
y2_knn5=knn5.predict(X_test_scaled)
y2_knn5=list(y2_knn5)

In [49]:
# Evaluating model on test data
mse = mean_squared_error(y_test, y2_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_test_scaled,y_test)}")

The RMSE is: 17870.33430617026
The R-squared is 0.4262081229189164


# KNN 2

In [50]:
# Constructing model
n_neighbors=2
knn2=KNeighborsRegressor(n_neighbors,weights='uniform')
knn2.fit(X_train_scaled,y_train)
y1_knn2=knn2.predict(X_train_scaled)
y1_knn2=list(y1_knn2)

In [51]:
#Training
mse = mean_squared_error(y_train, y1_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_train_scaled,y_train)}")

The RMSE is: 9047.247860717202
The R-squared is 0.8251670494273503


In [ ]:
# Testing
y2_knn2=knn2.predict(X_test_scaled)
y2_knn2=list(y2_knn2)

In [53]:
mse = mean_squared_error(y_test, y2_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_test_scaled,y_test)}")

The RMSE is: 18468.38061632058
The R-squared is 0.3871606059952606


# KNN 12

In [ ]:
# Constructing Model
n_neighbors=12
knn12=KNeighborsRegressor(n_neighbors,weights='uniform')
knn12.fit(X_train_scaled,y_train)
y1_knn12=knn12.predict(X_train_scaled)
y1_knn12=list(y1_knn12)

In [ ]:
# Training
mse = mean_squared_error(y_train, y1_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_train_scaled,y_train)}")

The RMSE is: 13318.173767495262
The R-squared is 0.6211392069816959


In [ ]:
# Testing
y2_knn12=knn12.predict(X_test_scaled)
y2_knn12=list(y2_knn12)
mse = mean_squared_error(y_test, y2_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_test_scaled,y_test)}")

The RMSE is: 18145.66265432824
The R-squared is 0.40839108878942876


# SVC - Linear

In [58]:
# Constructing the model
from sklearn import svm
vect = svm.SVC(kernel='linear')
vect.fit(X_train_scaled,y_train)
y1_svc=vect.predict(X_train_scaled)
y1_svc=list(y1_svc)

In [59]:
# Training
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_train_scaled,y_train)}")

The RMSE is: 9286.168292988728
The R-squared is 0.36515850573001934


In [ ]:
# Testing
y2_svc=vect.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_test_scaled,y_test)}")

The RMSE is: 18288.157272633332
The R-squared is 0.27901785714285715


# SVC - RBF

In [61]:
# Constructing model
rbf_svc =svm.SVC(kernel='rbf')
rbf_svc.fit(X_train_scaled,y_train)


SVC()

In [62]:
# Training
y1_svc=rbf_svc.predict(X_train_scaled)
y1_svc=list(y1_svc)
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_train_scaled,y_train)}")


The RMSE is: 21289.91964927114
The R-squared is 0.32303914272957285


In [63]:
# Testing
y2_svc=rbf_svc.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_test_scaled,y_test)}")

The RMSE is: 24000.348769787255
The R-squared is 0.290625
